In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [14]:
# Define paths to training and testing data
train_dir = "pillModel/dataset/train"
test_dir = "pillModel/dataset/test"

# Parameters
img_height, img_width = 150, 150
batch_size = 32
epochs = 20

In [30]:
# Data augmentation and preprocessing for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Rescale testing data
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 16 images belonging to 4 classes.
Found 12 images belonging to 4 classes.


In [44]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(4, activation="softmax")  # 5 classes for the pills
])

In [45]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation="softmax")(x)  # 5 classes

model = Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

/var/folders/l0/j84r7nr54m784fbk6716xjj80000gn/T/ipykernel_18602/30048562.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.2500 - loss: 2.3183 - val_accuracy: 0.5000 - val_loss: 1.3832
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 1.7979 - val_accuracy: 0.6667 - val_loss: 1.1488
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3750 - loss: 2.2079 - val_accuracy: 0.6667 - val_loss: 0.7979
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6250 - loss: 0.9081 - val_accuracy: 0.7500 - val_loss: 0.5821
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 1.2896 - val_accuracy: 0.9167 - val_loss: 0.4854
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5625 - loss: 1.3815 - val_accuracy: 0.9167 - val_loss: 0.3894
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8125 - loss: 0.5827 - val_accuracy: 1.0000 - val_loss: 0.3215
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7500 - loss: 0.5860 - val_accuracy: 0.9167 - val_loss: 0.2864
Epoch 9/20
1/1 ━

In [46]:
# Optionally evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step - accuracy: 0.9167 - loss: 0.2548
Test Accuracy: 91.67%


In [12]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Define the class labels
class_labels = ["2", "3", "4", "5"]

In [22]:
# Load and preprocess the image
pill2_img_path = "dataset/val/pill2.jpeg"
pill3_img_path = "dataset/val/pill3.jpeg"
pill4_img_path = "dataset/val/pill4.jpeg"
pill5_img_path = "dataset/val/pill5.jpeg"

img = load_img(pill2_img_path, target_size=(img_height, img_width))  # Use the same size as during training
img_array = img_to_array(img)  # Convert the image to a NumPy array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize to [0, 1] (same as during training)

# Make the prediction
predictions = model.predict(img_array)
predicted_index = np.argmax(predictions)  # Get the index of the highest probability
predicted_label = class_labels[predicted_index]
print(predicted_label)